# AI Project 5 - Decision Tree
## Jesse Brizzi, Geoffrey Churchill  

This is a printout of an IPython Notebook for Project 5.          
The original [python project](https://github.com/GeoffChurch/BinaryDecisionTree/) to run this is included with the submssion, and the original IPython notebook file can be downloaded [here](https://github.com/GeoffChurch/BinaryDecisionTree/edit/master/Decision%20Tree.ipynb).

### Intro
In this project we create a decision tree generator to predict when customers will leave a company's website. This notebook will outline our code structure, walk you through each step of the project, and complete with a detailed analysis of the performance on the provided data.

### Code

In [1]:
from collections import defaultdict,Counter
from bisect import bisect_right
# numpy and scipy are available from the anaconda collection:
#continuum.io/downloads
from numpy import log2
from scipy.stats import chi2
try:
	import cPickle as pickle
except:
	import pickle

We store our finished tree as a fractally-nested list. For example,

In [2]:
tree=[(23,[3,7]),'0',[(7,[2]),'0','1'],'1']

would represent the following decision tree:

In [3]:
def classify(element):
    attrVal = element.getAttr(23)
    if attrVal<3:
        return 0
    elif attrVal<7:
        attrVal=element.getAttr(7)
        if attrVal<2:
            return 0
        else:
            return 1
    else:
        return 1

However, while the tree is being grown, each leaf is a list of elements, rather than a label ('1' or '0'). For example,

In [4]:
tree=[(23,[3,7]),[('0',[2,4,3,3]),('0',[6,4,1,5]),('1',[1,2,3,4])]]

When we've finished growing, we simply change each leaf node to the mode of the labels of its elements. This is done using the following function:

In [5]:
"""
sets a node equal to the most common classification label among its elements
"""
def setClassifiers(parent,index):
	node=parent[index]
	if isLeaf(node):
		parent[index]=Counter(element[0] for element in node).most_common(1)[0][0]
	else:
		for index in xrange(len(node)-1):
			setClassifiers(node,index+1)

The core of our implementation is the following two methods, grow and split.

In [6]:
"""
given a pointer to a node, recursively splits it until its entropy is below the maximum allowed entropy
"""
def grow(self,parent,pIndex):
	node=parent[pIndex]
	if isLeaf(node): # if it's a leaf node, try to split it
		entropy=getEntropy(node)
		if(entropy>self.maxEntropy):
			self.split(parent,pIndex,entropy)
		else:
			print 'leaf entropy is low enough:',str(entropy),'<=',str(self.maxEntropy)
	else: # if it's an internal node, recurse on all children
		for index in xrange(len(node)-1):
			self.grow(node,index+1)

In [7]:
"""
given a pointer to a node, and its current entropy, attempts to split it on the best possible attribute and value.
the current implementation does a binary split, but this can easily be changed to support n-ary splits
"""
def split(self,parent,pIndex,minEntropy):
	node=parent[pIndex]
	print '\nAttempting to split leaf (size:',len(node),'entropy:'+str(minEntropy)+')'
	minTup=None
	for attr in xrange(len(node[0][1])): # for each attribute
		node.sort(key=lambda sample:sample[1][attr]) # sort the elements on that attribute
		currVal=node[0][1][attr]
		for index,(_,sample) in enumerate(node):
			newVal=sample[attr]
			if not newVal is currVal: # and every time the attribute's value changes we simulate a split and determine the subsequent total entropy
				currVal=newVal
				currEntropy,lChild,rChild=entropyBelow(minEntropy,node,attr,sample[attr],index)
				if currEntropy<minEntropy: # if the new entropy is lower than our previous min, save the split
					print 'split on attribute',self.attrNames[attr] if self.attrNames else attr,'at value',str(sample[attr])+': entropy='+str(currEntropy)
					minEntropy=currEntropy
					minTup=(attr,sample[attr],lChild,rChild)
	if not minTup:
		print 'Leaf is homogenous with value',map(lambda (attr,val): self.attrNames[attr]+': '+str(val),enumerate(node[0][1])) if self.attrNames else node[0][1],'and entropy',minEntropy
		return
	minAttr,splitpoint,leftChild,rightChild=minTup
	print 'size of children:'
	print '\tleft:',len(leftChild)
	print '\tright:',len(rightChild)
	pval=pvalue(node,[leftChild,rightChild])
	print 'pValue:',pval
	if pval>=self.maxP: # if our split isn't likely to lead to another statistically significant difference, we give up
		print 'Stop splitting!'
	else: # otherwise, perform the actual split and recurse
		parent[pIndex]=[(minAttr,[splitpoint]),leftChild,rightChild]
		self.grow(parent,pIndex)

To determine when and when not to split, we use entropy and p-value measurements.

The current entropy of a node, used by grow, is calculated by getEntropy.

In [8]:
"""
determines the entropy of elements with respect to their classification labels
"""
def getEntropy(elements):
	size=float(len(elements))
	counter=Counter(element[0] for element in elements)
	entropy=0
	for _,val in counter.iteritems():
		prob=val/size
		entropy-=prob*log2(prob)
	return entropy

The entropy of a node after simulating a split, used by...split, is calculated by entropyBelow.

In [9]:
"""
determines the entropy of the specified *SORTED* node after undergoing a split
on attrIndex at the value attrVal.
"""
def entropyBelow(maxEntropy,elements,attrIndex,attrVal,splitIndex):
	total=float(len(elements))
	#elements is already sorted so we just search for the first occurrence
	splitIndex=binaryFirstOccurence(attrIndex,attrVal,elements,0,splitIndex)
	#get left-child entropy
	subseqL=elements[:splitIndex]
	subtotal=len(subseqL)
	size=float(subtotal)
	counter=Counter([element[0] for element in subseqL])
	lEntropy=0
	for (_,val) in counter.iteritems():
		prob=val/size
		lEntropy-=prob*log2(prob)
	lEntropy*=subtotal/total
	if lEntropy>=maxEntropy: # if we've already surpassed maxEntropy, we fail early
		return (lEntropy,None,None)
	#get right-child entropy
	subseqR=elements[splitIndex:]
	subtotal=len(subseqR)
	size=float(subtotal)
	counter=Counter([element[0] for element in subseqR])
	rEntropy=0
	for (_,val) in counter.iteritems():
		prob=val/size
		rEntropy-=prob*log2(prob)
	return ((rEntropy*subtotal/total)+lEntropy,subseqL,subseqR)

The p-value of a split is calculated by pvalue.

In [10]:
"""
derives the p-value of splitting parent into leaves using the chi-squared test
"""
def pvalue(parent,leaves):
	#derive chi-squared statistic
	chiT=0
	n=float(len(parent))
	parentCounter=Counter(element[0] for element in parent)
	for leaf in leaves:
		ratio=len(leaf)/n
		for attr,leafCount in Counter(element[0] for element in leaf).iteritems():
			expected=parentCounter[attr]*ratio
			chiT+=((expected-leafCount)**2)/expected
	print 'chiT:',chiT
	#derive p-value
	return chi2.sf(chiT,len(parentCounter)-1)

###Results
For these results, we've kept the maxEntropy paramenter constant at 1, to demonstrate the effects of maxP.

####maxP=1.00
internal nodes: 8118<br>
leaf nodes: 8119<br>
total: 16237<br>
performance: 0.63352

####maxP=0.05
internal nodes: 337<br>
leaf nodes: 338<br>
total: 675<br>
performance: 0.71156

####maxP=0.01
internal nodes: 152<br>
leaf nodes: 153<br>
total: 305<br>
performance: 0.72572

####maxP=0.001
internal nodes: 78<br>
leaf nodes: 79<br>
total: 157<br>
performance: 0.73404

####maxP=0.0001
internal nodes: 46<br>
leaf nodes: 47<br>
total: 93<br>
performance: 0.72824

###Conclusion
When maxP is equal to 1, we don't stop splitting until every node is homogenous either in classifications or attributes. Because each node ends up with only a few elements, we lose some of the "big picture" of the data we're trying to classify, which leads to overfitting.<br>
Lowering maxP means that we'll only split a node if the split would result in a highly unexpected (non-trivial) distribution. The resultant nodes are more populous, so we have a clearer picture of the macro features of our data. Changing the p-value threshold is akin to focusing a lens to compromise between details (foreground) and patterns (background).<br>
<br>
We found that the tree performed best when maxP was about 0.001, which indicates that that is the appropriate level of generality for this dataset. Below that, the performance began to drop, indicating the complement of overfitting: underfitting. Underfitting occurs when we ignore aspects of the data that would have helped us in general.<br>
<br>
Because we prune our tree as we grow it, we're heuristically narrowing our search space to a very small subset based on naive probability measurements. Our tree would be more performant if we instead grew it completely and then pruned away extraneous nodes starting from the bottom, because we'd be pruning over the whole tree, rather than on a node-by-node basis, and because our judge of whether a branch is worthwhile would be based on its actual performance, rather than on how surprising it was.
